https://qiita.com/umi_mori/items/757834e0ef75f38cea19


In [1]:
! pip install pyserial

In [2]:
!  ls -l /dev/tty.*

crw-rw-rw-  1 root  wheel  0x900000a Feb  2 17:49 /dev/tty.Bluetooth-Incoming-Port
crw-rw-rw-  1 root  wheel  0x9000008 Feb  2 17:49 /dev/tty.ESP32test2
crw-rw-rw-  1 root  wheel  0x9000002 Feb  2 17:49 /dev/tty.JBLXtreme3
crw-rw-rw-  1 root  wheel  0x9000006 Feb  2 17:49 /dev/tty.NorisPowerbeats
crw-rw-rw-  1 root  wheel  0x9000004 Feb  2 17:49 /dev/tty.NoriyukiFujimurasSoloPro
crw-rw-rw-  1 root  wheel  0x9000000 Feb  2 17:48 /dev/tty.wlan-debug


In [ ]:
import serial

serialCommand = "test"
writeSer = serial.Serial('/dev/tty.Bluetooth-Incoming-Port',9600, timeout=3)
writeSer.write(serialCommand.encode())
writeSer.close()